# School Timetable Generator
This notebook uses constraint programming to assign classes to groups and teachers, fulfilling the following restrictions:

* Each subject must be taught the specified number of weekly hours.
* A teacher cannot teach two classes at the same time.
* Teachers cannot exceed their maximum assigned weekly hours.
* The maximum number of hours per day for each subject and group is limited.
* If a subject is taught more than one hour per day in a group, the hours must be consecutive.

## Common definitions

In [ ]:
from ortools.sat.python import cp_model

# Create test data
class Subject:
    def __init__(self, id, name, course, weekly_hours=5, max_hours_per_day=1):
        self.id = id
        self.course = course
        self.name = name
        self.weekly_hours = weekly_hours
        self.max_hours_per_day = max_hours_per_day
    def __repr__(self):
        return f"{self.name} ({self.course})"

class Teacher:
    def __init__(self, id, name, max_hours_week=20, subjects=[]):
        self.id = id
        self.name = name
        self.max_hours_week = max_hours_week
        self.subjects = subjects
    def __repr__(self):
        return f"{self.name}, {self.max_hours_week} h/w, {[s.id for s in self.subjects]}"

## Timetable, Subjects and Teachers

In [ ]:
# Configure the time table
num_days = 5
num_hours = 5

# groups
all_groups = ["1-A", "1-B"]

# subjects
math1 = Subject(id="math1", name="Maths", course="1", weekly_hours=10, max_hours_per_day=2)
eng1 = Subject(id="eng1", name="English", course="1", weekly_hours=10, max_hours_per_day=2)
all_subjects = [math1,eng1]

# teachers
John = Teacher(id=1, name="John", max_hours_week=20, subjects=[math1])
Jane = Teacher(id=2, name="Jane", max_hours_week=20, subjects=[eng1])
all_teachers = [John, Jane]
    
print("Groups:", all_groups)
print("Subjects:", [repr(s) for s in all_subjects])
print("Teachers:", [repr(t) for t in all_teachers])

## Model & decission variables

In [ ]:
# Initialize model
model = cp_model.CpModel()

# Create decision variables (group-subject-teacher-day-hour)
assignments = {}  
for group in all_groups:
    course = group.split('-')[0]
    for subject in all_subjects:
        if subject.course == course:
            for teacher in all_teachers:
                if subject in teacher.subjects:
                    for d in range(num_days):
                        for h in range(num_hours):
                            key = (group, subject.id, teacher.id, d, h)
                            assignments[key] = model.NewBoolVar(f"g:{group} sub:{subject.id} t:{teacher.name} d:{d} h:{h}")


## Restrictions

In [ ]:

# Each subject must be taught the specified weekly hours
for group in all_groups:
    course = group.split('-')[0]
    for subject in all_subjects:
        if subject.course == course:
            model.Add(sum(assignments[key] for key in assignments if key[0] == group and key[1] == subject.id) == subject.weekly_hours)

# A teacher cannot teach two classes at the same time
for teacher in all_teachers:
    for d in range(num_days):
        for h in range(num_hours):
            model.AddAtMostOne(assignments[key] for key in assignments if key[2] == teacher.id and key[3] == d and key[4] == h)

# Teachers cannot exceed their maximum assigned weekly hours.
for teacher in all_teachers:
    max_hours = teacher.max_hours_week
    teacher_total_hours = sum(assignments[key] for key in assignments if key[2] == teacher.id)
    model.Add(teacher_total_hours <= max_hours)
    
# The maximum number of hours per day for each subject and group is limited.
for group in all_groups:
    course = group.split('-')[0]
    for subject in all_subjects:
        if subject.course == course:
            for teacher in all_teachers:
                if subject in teacher.subjects:
                    for d in range(num_days):
                        # Get hours for this specific group-subject-teacher combination
                        hour_vars = [assignments[key] for key in assignments  if key[0] == group and key[1] == subject.id and key[2] == teacher.id and key[3] == d]
                        model.Add(sum(hour_vars) <=subject.max_hours_per_day)

The following cell is commented because it is not working as expected

In [ ]:
#If a subject is taught more than one hour per day in a group, the hours must be consecutive.

for group in all_groups:
    course = group.split('-')[0]
    for subject in all_subjects:
        if subject.course == course:
            for d in range(num_days):
                # 1) create aggregated variables y_h
                y_vars = []
                for h in range(num_hours):
                    y = model.NewBoolVar(f"y_{group}_{subject.id}_d{d}_h{h}")  # y[group,subject,day,h]
                    # Link y with the assignment variables (sum over teachers)
                    assign_vars = [
                        assignments[key]
                        for key in assignments
                        if key[0] == group and key[1] == subject.id and key[3] == d and key[4] == h
                    ]
                    if assign_vars:
                        # Equality: y == sum(assign_vars). (Assumes no two teachers simultaneously for the same group-subject-slot.)
                        model.Add(sum(assign_vars) == y)
                    else:
                        # if there are no possible teachers for that slot, force y == 0
                        model.Add(y == 0)
                    y_vars.append(y)

                # 2) define starts: start_h = 1 if y_h == 1 and y_{h-1} == 0
                starts = []
                for h in range(num_hours):
                    s = model.NewBoolVar(f"start_{group}_{subject.id}_d{d}_h{h}")
                    starts.append(s)
                    if h == 0:
                        # start at h=0 <=> y_0
                        model.Add(s == y_vars[0])
                    else:
                        # linearization of s == y_h & (not y_{h-1}):
                        #  s >= y_h - y_{h-1}
                        #  s <= y_h
                        #  s <= 1 - y_{h-1}
                        model.Add(s >= y_vars[h] - y_vars[h-1])
                        model.Add(s <= y_vars[h])
                        model.Add(s <= 1 - y_vars[h-1])

                # 3) at most one block start per day -> ensures a single contiguous block (or none)
                model.Add(sum(starts) <= 1)



## Solving the problem

In [ ]:
# Solve the model and display the timetable
from IPython.display import display, Markdown

solver = cp_model.CpSolver()
status = solver.Solve(model)

# Create and display markdown timetable
if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    for group in all_groups:
        course = group.split('-')[0]
        markdown_text = [f"## Schedule for {group}\n"]
        markdown_text.append("| Hour | Monday | Tuesday | Wednesday | Thursday | Friday |")
        markdown_text.append("|------|---------|----------|-----------|-----------|---------|")
        
        for h in range(num_hours):
            row = [f"Hour {h}"]
            for d in range(num_days):
                cell_content = ""
                for subject in all_subjects:
                    if subject.course == course:
                        for teacher in all_teachers:
                            if subject in teacher.subjects:
                                key = (group, subject.id, teacher.id, d, h)
                                if solver.Value(assignments[key]) == 1:
                                    cell_content = f"{subject.name}<br>{teacher.name}"
                row.append(cell_content if cell_content else "-")
            markdown_text.append(f"| {' | '.join(row)} |")
        
        # Display the timetable as rendered markdown
        display(Markdown('\n'.join(markdown_text)))
else:
    print("No feasible solution found.")